In [25]:
ss=pd.read_csv('task3a.csv')
ss.head()

,public_id,predicted_rating
0,81a67c96,partially false
1,6e5ec6fb,false
2,d9cd4895,false
3,4a1a9b9f,false
4,6d16fa40,false


In [26]:
from sklearn.metrics import classification_report

In [27]:
true=pd.read_csv('Task3a_testing.csv')
true.head()

,public_id,text,title,our rating
0,81a67c96,Former state House Majority Leader Adam Hasner...,- The Washington Post,partially false
1,6e5ec6fb,"Editor’s note: Subsequent to this article, VTD...",Rubio Comments on Iran Nuclear Deal,false
2,d9cd4895,The hypocritical Lib Dems want to ignore the r...,Climate Alarmists Caught Manipulating Temperat...,false
3,4a1a9b9f,Urgent action to boost the number of children ...,Who are the arsonists setting rural fires in W...,false
4,6d16fa40,By Ken Allen AFSCME Council 75 represents the ...,"Diabetes prescriptions now cost NHS £1bn, figu...",false


In [28]:
print(classification_report(true['our rating'],pred))

                 precision    recall  f1-score   support

          false       0.53      0.80      0.64       113
          other       0.49      0.56      0.52        41
partially false       0.66      0.45      0.54       141
           true       0.38      0.28      0.32        69

       accuracy                           0.54       364
      macro avg       0.51      0.52      0.50       364
   weighted avg       0.55      0.54      0.53       364



In [29]:
from pathlib import Path

In [30]:
import re, os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from tkinter import Tk, filedialog
from IPython.display import Markdown

duplicates = ["1a46b040", "c1e124d5", "8209ca7b", "e2abfbe6", "39f5c37f",
              "47423bb6", "097c142a", "08bc59f4", "af3393ce", "a39d07df"]

"""## Gold Standard"""

root = Tk()
root.withdraw() #dont show base window
path_to_test_data = Path('Task3a_testing.csv')#filedialog.askopenfile() #file selection dialog
print(path_to_test_data.name) #check, that the correct file has been selected

#depending on whether task a or task b testing data has been loaded
column_name_test = "our rating" if "3a" in path_to_test_data.name else "category"
column_name_system = "predicted_rating" if "3a" in path_to_test_data.name else "predicted_domain"

#read gold standard
#If you don't wan't to use a dialog, specify the path directly
test_data = pd.read_csv(path_to_test_data.name, encoding="UTF-8")
test_data.head() #quick check, if the data has been correctly read.


Task3a_testing.csv


,public_id,text,title,our rating
0,81a67c96,Former state House Majority Leader Adam Hasner...,- The Washington Post,partially false
1,6e5ec6fb,"Editor’s note: Subsequent to this article, VTD...",Rubio Comments on Iran Nuclear Deal,false
2,d9cd4895,The hypocritical Lib Dems want to ignore the r...,Climate Alarmists Caught Manipulating Temperat...,false
3,4a1a9b9f,Urgent action to boost the number of children ...,Who are the arsonists setting rural fires in W...,false
4,6d16fa40,By Ken Allen AFSCME Council 75 represents the ...,"Diabetes prescriptions now cost NHS £1bn, figu...",false


In [31]:
"""## Participants file"""

#read participant data 

#If you don't wan't to use a dialog, specify the path directly
path_to_system_predictions = Path('Task3a.csv')#filedialog.askopenfile() #file selection dialog
print(path_to_system_predictions.name) #check, if the correct file has been selected

if path_to_system_predictions.name.endswith("tsv"):
    system_predictions = pd.read_csv(path_to_system_predictions.name, encoding="utf-8", sep="\t") 
else:
    system_predictions = pd.read_csv(path_to_system_predictions.name, encoding="utf-8")

#Check, if the number of entries is correct
print(len(system_predictions))
system_predictions.head()



Task3a.csv
364


,public_id,predicted_rating
0,81a67c96,partially false
1,6e5ec6fb,false
2,d9cd4895,false
3,4a1a9b9f,false
4,6d16fa40,false


In [15]:
"""## Drop duplicates"""

#Only for 3A
system_predictions_no_duplicates = system_predictions[~system_predictions["public_id"].isin(duplicates)]
if(len(system_predictions) - len(system_predictions_no_duplicates)==10):
    print("Dropping duplicates was successful") 
    system_predictions = system_predictions_no_duplicates.reset_index(drop=True)
else:
    print("Dropping duplicates was NOT successful. Please mark for a later check.")
    print(f"system_predictions has {len(system_predictions)} entries and system_predictions_no_duplicates has {len(system_predictions_no_duplicates)} entries.")
    
print(f"system_predictions has {len(system_predictions)} entries")


Dropping duplicates was successful
system_predictions has 354 entries


In [32]:
"""## Check public ids"""

#check, if the id columns are identical
#there are some public ids, that are saved differently depending on the editor used to view the file 
#    test_data    system_predictions (example for public ids that might be problematic)
#    public_id    public_id
#86   080e0024       8E+025
#129  17334e19  1,7334E+023
#210  84231e18  8,4231E+022
#219  85100e77    8,51E+081
#271  04866616      4866616
#272  03963580      3963580
#363  02695016      2695016
#If you can verify that the ids are identical, than you can proceed with the evaluation. If not, please inspect further.
print(test_data["public_id"].isin(system_predictions["public_id"]).value_counts())
matched = pd.DataFrame(test_data["public_id"].eq(system_predictions["public_id"]))
if len(matched[matched["public_id"]==False]) == 0:
    display(Markdown("<span style='color: #ff0000'>everything is fine, please proceed with the evaluation.</span>".upper()))
else:
    try:
        print(pd.concat((
            test_data["public_id"].loc[
                test_data["public_id"].where(test_data["public_id"]==system_predictions["public_id"]).isna()], 
            system_predictions["public_id"].loc[
                system_predictions["public_id"].where(system_predictions["public_id"]==test_data["public_id"]).isna()]), axis =1))
    except ValueError as e:
        display(Markdown("<span style='color: #ff0000'>The number of entries in both files is different. An evaluation is not possible.</span>".upper())) 
        merged_dataframe = pd.merge(test_data["public_id"], system_predictions["public_id"], how="outer", indicator=True)
        print("entries missing in the participants file:\n", 
              merged_dataframe[merged_dataframe["_merge"]=="left_only"]["public_id"])
        print("\nentries missing in the systems file:\n", system_predictions["public_id"].loc[
        system_predictions["public_id"].isin(merged_dataframe[merged_dataframe["_merge"]=="right_only"]["public_id"])])


True    364
Name: public_id, dtype: int64


<SPAN STYLE='COLOR: #FF0000'>EVERYTHING IS FINE, PLEASE PROCEED WITH THE EVALUATION.</SPAN>

In [33]:
"""## Evaluation"""

try:
    classification_report_results = classification_report(test_data[column_name_test], system_predictions[column_name_system].str.lower().str.strip(), output_dict=True)
    eval_results = pd.DataFrame.from_dict(classification_report_results).transpose()
    participant_file = re.sub(".+/(\d+)/*[\w_-]*/([\w_-]+)(\.(tsv|csv))?", "\\1_\\2", path_to_system_predictions.name)

    eval_results.to_csv("classification_report_" + participant_file + ".csv")
    #pretty print
    print(classification_report(test_data[column_name_test], system_predictions[column_name_system].str.lower().str.strip(), digits=10))
except KeyError as e:
    print("The column names of the participant file don't match the required column names ('public_id, 'predicted_rating'):", 
          system_predictions.columns)

#Destroy the "invisible" root window
#root.destroy()


                 precision    recall  f1-score   support

          false  0.5294117647 0.7964601770 0.6360424028       113
          other  0.4893617021 0.5609756098 0.5227272727        41
partially false  0.6597938144 0.4539007092 0.5378151261       141
           true  0.3800000000 0.2753623188 0.3193277311        69

       accuracy                      0.5384615385       364
      macro avg  0.5146418203 0.5216747037 0.5039781332       364
   weighted avg  0.5470832061 0.5384615385 0.5251927360       364

